This notebook demonstrates how to implement a reform in OpenFisca.
The reform is dummy and has no other purpose than to be a tutorial.

# The big picture

Creating a reform in OpenFisca is pretty simple: we just need to write a function named `build_reform` which receives the tax_benefit_system parameter and returns a `Reform` object. We can work in this IPython Notebook, there is no need for the moment to install OpenFisca.

The `tax_benefit_system` parameter is an instance of a `TaxBenefitSystem` object which holds the legislation and the entities of the country. The legislation is a JSON data structure containing the parameters of the law, like thresholds, tax scales, etc. The entities are objects pointing to formulas.

The inner contents of the `build_reform` function is the code of the reform. In general it does two things: change the legislation and/or change formulas (via the entities). By change we mean add, delete or replace.

Here is an empty skeleton example:

    def build_reform(tax_benefit_system):

        # Update legislation (skipped...)

        # Update formulas (skipped...)

        return reforms.Reform(
            name=u'Dummy reform',
            reference=tax_benefit_system,
            # Here we'll fill other attributes later
            )

The actual contents of the `build_reform` function will be explained below. Let's just finish with the big picture.

This `build_reform` function can be called from the outside like this (this is what we'll do after having finished writing it):

    dummy_reform = build_reform(tax_benefit_system)  # tax_benefit_system was initialized above.
    scenario = dummy_reform.new_scenario()  # Declare a scenario (scenario contents skipped...)
    reference_simulation = scenario.new_simulation(reference=True)
    reform_simulation = scenario.new_simulation(debug=True)

We just declared a reference and a reform simulation object. Now we can calculate the same formula (for example "revdisp") with and without the reform:

    reference_revdisp = reference_simulation.calculate('revdisp')
    reform_revdisp = reform_simulation.calculate('revdisp')
    print(reference_revdisp, reform_revdisp)

More interesting: we can plug our reform on the OpenFisca API and watch the results interactively in the OpenFisca "demonstrator" (http://ui.openfisca.fr/). The reform can be added to the reforms list on the OpenFisca website too (http://www.openfisca.fr/reformes).

_This should be the first goal of a good reform author :-)_

Now let's dive deeper into the techniques of writing reforms.

In [1]:
# First, import required and useful Python modules.
# You can ignore this cell.

import copy
import datetime
import json

from IPython.display import display
from IPython.html import widgets
from IPython.lib.pretty import pprint
from IPython.utils.traitlets import Unicode
from openfisca_core import columns, conv, formulas, legislationsxml, reforms
import openfisca_france
from openfisca_france import entities, model

Now let's create the french tax-benefit system.

In [2]:
TaxBenefitSystem = openfisca_france.init_country()
tax_benefit_system = TaxBenefitSystem()

# Change the legislation

In OpenFisca the legislation is organized as a tree and is stored in a XML file.

To change the legislation, we can either change one or many existing nodes, add new nodes or delete existing nodes.

The idea is to duplicate the reference legislation (the existing one) with `copy.deepcopy` Python function, then applying our changes. Reminder: the reference legislation is accessible as an attribute of the `tax_benefit_system` object.

For the purpose of this dummy reform, we are going to change the first slice of the "IRPP".

First, download the reference legislation XML file from the OpenFisca-France source code (save it in a file on your computer): https://raw.githubusercontent.com/openfisca/openfisca-france/master/openfisca_france/param/param.xml

Name it like `param-<reform_name>.xml`.

Then modify the XML file with your preferred tool (Notepad++ with JSTool or xmlcopyeditor for example).

Once finished, please run the cells below to add the file upload utility to this notebook, then upload your new XML file.

In [3]:
# Just run this cell with Ctrl-Enter

class FileWidget(widgets.DOMWidget):
    _view_name = Unicode('FilePickerView', sync=True)
    value = Unicode(sync=True)
    filename = Unicode(sync=True)
    
    def __init__(self, **kwargs):
        widgets.DOMWidget.__init__(self, **kwargs) # Call the base.
        self.errors = widgets.CallbackDispatcher(accepted_nargs=[0, 1])
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)


In [4]:
%%javascript

// Just run this cell with Ctrl-Enter

require(["widgets/js/widget"], function(WidgetManager){
    var FilePickerView = IPython.WidgetView.extend({
        render: function() {
            this.setElement($('<input />').attr('type', 'file'));
        },
        events: {'change': 'handle_file_change'},
        handle_file_change: function(evt) { 
            var file = evt.target.files[0];
            if (file) {
                var that = this;
                var file_reader = new FileReader();
                file_reader.onload = function(e) {
                    that.model.set('value', e.target.result);
                    that.touch();
                }
                file_reader.readAsText(file);
            } else {
                this.send({ 'event': 'error' });
            }
            this.model.set('filename', file.name);
            this.touch();
        },
    });
    WidgetManager.register_widget_view('FilePickerView', FilePickerView);
});

In [5]:
# Just run this cell with Ctrl-Enter

reform_legislation_json = None
file_widget = FileWidget()
def file_loading():
    print("Loading %s" % file_widget.filename)
file_widget.on_trait_change(file_loading, 'filename')
def file_loaded():
    global reform_legislation_json
    reform_legislation_xml = file_widget.value
    reform_legislation_json, error = legislationsxml.xml_legislation_str_to_json(reform_legislation_xml)
    print(
        u'XML file loaded successfully' if error is None else u'XML file loading has failed: {}'.format(error)
        )
file_widget.on_trait_change(file_loaded, 'value')
def file_failed(*args):
    print("Could not load file contents of %s" % file_widget.filename)
file_widget.errors.register_callback(file_failed)
file_widget

Loading param-cbenz.xml
XML file loaded successfully


Now click on the "Choose File" button just above and upload your XML file.

The variable `reform_legislation_json` should contain your modified legislation if the file was loaded successfully.

_Explanation: it is called "json" because OpenFisca internals store the legislation in JSON, not XML. The conversion was done automatically._

You can repeat the upload step if you change the XML file on your computer. In this case, execute again the cells following the file upload to update the variables.

Let's declare a concrete `build_reform_1` function using the `reform_legislation_json` variable we just created.

In [15]:
def build_reform_1(tax_benefit_system):
    return reforms.Reform(
        entity_class_by_key_plural = entities.entity_class_by_key_plural,  # Keep the reference entities.
        legislation_json = reform_legislation_json,  # Was generated from the XML file you uploaded.
        name = u'Dummy reform 1 (legislation only)',
        reference = tax_benefit_system,
        )

We can try this reform on a scenario with a 40 years old person earning 5000€ a year:

In [7]:
dummy_reform_1 = build_reform_1(tax_benefit_system)  # tax_benefit_system was initialized above.

scenario_1 = dummy_reform_1.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = datetime.date(1974, 1, 1),
        sali = 5000,
        ),
    )

reference_simulation_1 = scenario_1.new_simulation(reference=True)
reform_simulation_1 = scenario_1.new_simulation()

# Calculate the "Revenu disponible" for both reference and reform simulations.
reference_revdisp_1 = reference_simulation_1.calculate('revdisp')
reform_revdisp_1 = reform_simulation_1.calculate('revdisp')

display('reference value', reference_revdisp_1, 'reform value', reform_revdisp_1)


'reference value'

array([ 8932.73144531], dtype=float32)

'reform value'

array([-3491067.75], dtype=float32)

You may notice a difference if the scenario matches a part of the legislation params you changed.

# Change a formula

In OpenFisca a formula is a Python class with these attributes and methods:

* `column`: type of the computed value(s)
* `entity_class`: class of the entity on which the computation is defined
* `label`: string describing the formula (optional, but recommended).
* `url`: external web page describing the formula (optional, but recommended).
* `function()`: the actual code of the computation. The parameters are provided automatically by the core engine:
    * `simulation` is the `Simulation` object instance allowing to compute dependent variables
    * `period` is the period asked by the caller
    * The function returns a tuple with 2 values: the output period and the result.

_You may ask a developer for further information about the periods system._

The formula is defined as vectorial computations (using Python NumPy). Each variable is an `Array` object and might contain more than a single value. Specifically the `Array` will contain as much values as the number of persons in the entity.

Existing variable for "Revenu net de l'individu" example:

    class revenu_net_individu(formulas.SimpleFormulaColumn):
        column = columns.FloatCol
        entity_class = entities.Individus
        label = u"Revenu net de l'individu"

        def function(self, simulation, period):
            period = period.start.offset('first-of', 'year').period('year')
            pen = simulation.calculate('pen', period)
            rev_cap = simulation.calculate('rev_cap', period)
            rev_trav = simulation.calculate('rev_trav', period)

            return period, pen + rev_cap + rev_trav

In order to change a variable, copy its class and change what you want.

_Very-technical note: forget about inheriting classes in this case, since "metaclasses" are used._

Here we'll just change the `function` function, returning only the content of `rev_trav`.

In [8]:
from openfisca_france.model.common import revenu_net_individu

class revenu_net_individu_2(formulas.SimpleFormulaColumn):
    column = columns.FloatCol
    entity_class = entities.Individus
    label = u"Revenu net de l'individu"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'year').period('year')
        rev_trav = simulation.calculate('rev_trav', period)
        return period, rev_trav

This time let's declare a new `build_reform_2` which doesn't change the legislation but changes the `revenu_net_individu`.

In order to change a variable formula we need to change the entity related to the variable.
There is a helper to clone entities: the `reforms.clone_entity_classes` function.

We have to replace the variable in an attribute of the entity called `column_by_name`.

Here it is:

In [9]:
def build_reform_2(tax_benefit_system):
    reform_entity_class_by_key_plural = reforms.clone_entity_classes(entities.entity_class_by_key_plural)

    # Change the ReformIndividus entity "revenu_net_individu" variable.
    ReformIndividus = reform_entity_class_by_key_plural['individus']
    ReformIndividus.column_by_name['revenu_net_individu'] = revenu_net_individu_2

    return reforms.Reform(
        entity_class_by_key_plural = reform_entity_class_by_key_plural,
        legislation_json = tax_benefit_system.legislation_json,  # Keep the reference legislation.
        name = u'Dummy reform 2 (formula only)',
        reference = tax_benefit_system,
        )

Again, the test:

In [10]:
dummy_reform_2 = build_reform_2(tax_benefit_system)  # tax_benefit_system was initialized above.

scenario_2 = dummy_reform_2.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = datetime.date(1974, 1, 1),
        sali = 5000,
        ),
    )

reference_simulation_2 = scenario_2.new_simulation(reference=True)
reform_simulation_2 = scenario_2.new_simulation()

# Calculate the "Revenu disponible" for both reference and reform simulations.
reference_revdisp_2 = reference_simulation_2.calculate('revdisp')
reform_revdisp_2 = reform_simulation_2.calculate('revdisp')

display('reference value', reference_revdisp_2, 'reform value', reform_revdisp_2)


'reference value'

array([ 8932.73144531], dtype=float32)

'reform value'

array([ 8932.73144531], dtype=float32)

# Change legislation and formulas

We can combine the 2 versions (`build_reform_1` and `build_reform_2`) in a single `build_reform` function which modifies both the legislation params and variable formula(s).

In [12]:
def build_reform(tax_benefit_system):
    reform_entity_class_by_key_plural = reforms.clone_entity_classes(entities.entity_class_by_key_plural)

    # Change the ReformIndividus entity "revenu_net_individu" variable.
    ReformIndividus = reform_entity_class_by_key_plural['individus']
    ReformIndividus.column_by_name['revenu_net_individu'] = revenu_net_individu_2

    return reforms.Reform(
        entity_class_by_key_plural = reform_entity_class_by_key_plural,
        legislation_json = reform_legislation_json,  # Was built from the XML file you uploaded.
        name = u'Dummy reform',
        reference = tax_benefit_system,
        )

Again, the test:

In [13]:
dummy_reform = build_reform(tax_benefit_system)  # tax_benefit_system was initialized above.

scenario = dummy_reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = datetime.date(1974, 1, 1),
        sali = 5000,
        ),
    )

reference_simulation = scenario.new_simulation(reference=True)
reform_simulation = scenario.new_simulation()

# Calculate the "Revenu disponible" for both reference and reform simulations.
reference_revdisp = reference_simulation.calculate('revdisp')
reform_revdisp = reform_simulation.calculate('revdisp')

display('reference value', reference_revdisp, 'reform value', reform_revdisp)


'reference value'

array([ 8932.73144531], dtype=float32)

'reform value'

array([-3491067.75], dtype=float32)

# Publish your reform

In order to make your work accessible to the others like the "OpenFisca demonstrator" (http://ui.openfisca.fr/) you might store it into a Git repository hosted for example on GitHub.

We won't explain how to do this here so please call a developer if needed.

Once your repository exists on GitHub, a developer will plug it to the OpenFisca API making it accessible to the demonstrator. Your reform will be added to the reforms list on the OpenFisca website (http://www.openfisca.fr/reformes).